In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import time
from models import  LinfPGDAttack, ResNet18

In [2]:
learning_rate = 0.1
epsilon = 0.0314
k = 7
alpha = 0.00784
file_name = 'pgd_adversarial_training'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = LinfPGDAttack(net,epsilon,alpha,k)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [5]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        adv = adversary.perturb(inputs, targets)
        adv_outputs = net(adv)
        loss = criterion(adv_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = adv_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current adversarial train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current adversarial train loss:', loss.item())

    print('\nTotal adversarial train accuarcy:', 100. * correct / total)
    print('Total adversarial train loss:', train_loss)

In [6]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            total += targets.size(0)

            outputs = net(inputs)
            loss = criterion(outputs, targets)
            benign_loss += loss.item()

            _, predicted = outputs.max(1)
            benign_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('\nCurrent batch:', str(batch_idx))
                print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current benign test loss:', loss.item())

            adv = adversary.perturb(inputs, targets)
            adv_outputs = net(adv)
            loss = criterion(adv_outputs, targets)
            adv_loss += loss.item()

            _, predicted = adv_outputs.max(1)
            adv_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

In [7]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [8]:
t1 = time.time()
for epoch in range(0, 50):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)
t2 = time.time()


[ Train epoch: 0 ]

Current batch: 0
Current adversarial train accuracy: 0.0
Current adversarial train loss: 4.878740310668945

Current batch: 10
Current adversarial train accuracy: 0.09375
Current adversarial train loss: 2.9484665393829346

Current batch: 20
Current adversarial train accuracy: 0.1328125
Current adversarial train loss: 2.475701332092285

Current batch: 30
Current adversarial train accuracy: 0.1171875
Current adversarial train loss: 2.486109495162964

Current batch: 40
Current adversarial train accuracy: 0.15625
Current adversarial train loss: 2.35545015335083

Current batch: 50
Current adversarial train accuracy: 0.140625
Current adversarial train loss: 2.336601495742798

Current batch: 60
Current adversarial train accuracy: 0.1484375
Current adversarial train loss: 2.305589199066162

Current batch: 70
Current adversarial train accuracy: 0.1171875
Current adversarial train loss: 2.341203451156616

Current batch: 80
Current adversarial train accuracy: 0.1796875
Current


Current batch: 120
Current adversarial train accuracy: 0.234375
Current adversarial train loss: 2.0166537761688232

Current batch: 130
Current adversarial train accuracy: 0.2109375
Current adversarial train loss: 2.0726959705352783

Current batch: 140
Current adversarial train accuracy: 0.21875
Current adversarial train loss: 2.076143980026245

Current batch: 150
Current adversarial train accuracy: 0.1875
Current adversarial train loss: 2.018078565597534

Current batch: 160
Current adversarial train accuracy: 0.28125
Current adversarial train loss: 2.043539524078369

Current batch: 170
Current adversarial train accuracy: 0.234375
Current adversarial train loss: 2.0736942291259766

Current batch: 180
Current adversarial train accuracy: 0.234375
Current adversarial train loss: 2.0854599475860596

Current batch: 190
Current adversarial train accuracy: 0.203125
Current adversarial train loss: 2.021531581878662

Current batch: 200
Current adversarial train accuracy: 0.25
Current adversaria


Current batch: 250
Current adversarial train accuracy: 0.25
Current adversarial train loss: 1.9546165466308594

Current batch: 260
Current adversarial train accuracy: 0.2890625
Current adversarial train loss: 2.0218608379364014

Current batch: 270
Current adversarial train accuracy: 0.28125
Current adversarial train loss: 1.9219988584518433

Current batch: 280
Current adversarial train accuracy: 0.21875
Current adversarial train loss: 2.040151596069336

Current batch: 290
Current adversarial train accuracy: 0.296875
Current adversarial train loss: 1.8316770792007446

Current batch: 300
Current adversarial train accuracy: 0.21875
Current adversarial train loss: 2.0278754234313965

Current batch: 310
Current adversarial train accuracy: 0.328125
Current adversarial train loss: 1.8789409399032593

Current batch: 320
Current adversarial train accuracy: 0.1953125
Current adversarial train loss: 2.0152945518493652

Current batch: 330
Current adversarial train accuracy: 0.3125
Current adversa


Current batch: 380
Current adversarial train accuracy: 0.3046875
Current adversarial train loss: 1.839961290359497

Current batch: 390
Current adversarial train accuracy: 0.3375
Current adversarial train loss: 1.797252893447876

Total adversarial train accuarcy: 28.51
Total adversarial train loss: 742.2918585538864

[ Test epoch: 3 ]

Current batch: 0
Current benign test accuracy: 0.55
Current benign test loss: 1.5110390186309814
Current adversarial test accuracy: 0.36
Current adversarial test loss: 1.8029682636260986

Current batch: 10
Current benign test accuracy: 0.5
Current benign test loss: 1.5278902053833008
Current adversarial test accuracy: 0.32
Current adversarial test loss: 1.8632234334945679

Current batch: 20
Current benign test accuracy: 0.45
Current benign test loss: 1.500819206237793
Current adversarial test accuracy: 0.33
Current adversarial test loss: 1.7803667783737183

Current batch: 30
Current benign test accuracy: 0.52
Current benign test loss: 1.4314390420913696



Current batch: 60
Current benign test accuracy: 0.39
Current benign test loss: 1.447192668914795
Current adversarial test accuracy: 0.29
Current adversarial test loss: 1.745882511138916

Current batch: 70
Current benign test accuracy: 0.49
Current benign test loss: 1.4702545404434204
Current adversarial test accuracy: 0.26
Current adversarial test loss: 1.8853715658187866

Current batch: 80
Current benign test accuracy: 0.5
Current benign test loss: 1.4571980237960815
Current adversarial test accuracy: 0.37
Current adversarial test loss: 1.8123565912246704

Current batch: 90
Current benign test accuracy: 0.44
Current benign test loss: 1.4146875143051147
Current adversarial test accuracy: 0.26
Current adversarial test loss: 1.7559717893600464

Total benign test accuarcy: 48.44
Total adversarial test Accuarcy: 32.06
Total benign test loss: 146.09417974948883
Total adversarial test loss: 180.7355796098709
Model Saved!

[ Train epoch: 5 ]

Current batch: 0
Current adversarial train accura

...

Current adversarial test accuracy: 0.51
Current adversarial test loss: 1.3687825202941895

Current batch: 80
Current benign test accuracy: 0.81
Current benign test loss: 0.7565192580223083
Current adversarial test accuracy: 0.49
Current adversarial test loss: 1.4207255840301514

Current batch: 90
Current benign test accuracy: 0.74
Current benign test loss: 0.7093693614006042
Current adversarial test accuracy: 0.44
Current adversarial test loss: 1.31264066696167

Total benign test accuarcy: 75.62
Total adversarial test Accuarcy: 47.63
Total benign test loss: 77.93828374147415
Total adversarial test loss: 141.50200772285461
Model Saved!

[ Train epoch: 45 ]

Current batch: 0
Current adversarial train accuracy: 0.4140625
Current adversarial train loss: 1.4246103763580322

Current batch: 10
Current adversarial train accuracy: 0.5703125
Current adversarial train loss: 1.1903305053710938

Current batch: 20
Current adversarial train accuracy: 0.5234375
Current adversarial train loss: 1.255069


Current batch: 70
Current adversarial train accuracy: 0.4609375
Current adversarial train loss: 1.2042567729949951

Current batch: 80
Current adversarial train accuracy: 0.546875
Current adversarial train loss: 1.1971871852874756

Current batch: 90
Current adversarial train accuracy: 0.5390625
Current adversarial train loss: 1.135561227798462

Current batch: 100
Current adversarial train accuracy: 0.5234375
Current adversarial train loss: 1.3648004531860352

Current batch: 110
Current adversarial train accuracy: 0.4609375
Current adversarial train loss: 1.2266921997070312

Current batch: 120
Current adversarial train accuracy: 0.5546875
Current adversarial train loss: 1.1203465461730957

Current batch: 130
Current adversarial train accuracy: 0.4921875
Current adversarial train loss: 1.2459228038787842

Current batch: 140
Current adversarial train accuracy: 0.53125
Current adversarial train loss: 1.3234546184539795

Current batch: 150
Current adversarial train accuracy: 0.5390625
Curre


Current batch: 190
Current adversarial train accuracy: 0.59375
Current adversarial train loss: 1.171142816543579

Current batch: 200
Current adversarial train accuracy: 0.484375
Current adversarial train loss: 1.3337972164154053

Current batch: 210
Current adversarial train accuracy: 0.4765625
Current adversarial train loss: 1.281655192375183

Current batch: 220
Current adversarial train accuracy: 0.484375
Current adversarial train loss: 1.3118432760238647

Current batch: 230
Current adversarial train accuracy: 0.5
Current adversarial train loss: 1.3203539848327637

Current batch: 240
Current adversarial train accuracy: 0.5625
Current adversarial train loss: 1.1814725399017334

Current batch: 250
Current adversarial train accuracy: 0.546875
Current adversarial train loss: 1.1344530582427979

Current batch: 260
Current adversarial train accuracy: 0.5234375
Current adversarial train loss: 1.2296943664550781

Current batch: 270
Current adversarial train accuracy: 0.5078125
Current advers


Current batch: 320
Current adversarial train accuracy: 0.5234375
Current adversarial train loss: 1.1743390560150146

Current batch: 330
Current adversarial train accuracy: 0.4921875
Current adversarial train loss: 1.3417387008666992

Current batch: 340
Current adversarial train accuracy: 0.546875
Current adversarial train loss: 1.1872016191482544

Current batch: 350
Current adversarial train accuracy: 0.5703125
Current adversarial train loss: 1.1117452383041382

Current batch: 360
Current adversarial train accuracy: 0.5703125
Current adversarial train loss: 1.1117045879364014

Current batch: 370
Current adversarial train accuracy: 0.515625
Current adversarial train loss: 1.2563291788101196

Current batch: 380
Current adversarial train accuracy: 0.3984375
Current adversarial train loss: 1.4509567022323608

Current batch: 390
Current adversarial train accuracy: 0.525
Current adversarial train loss: 1.2042522430419922

Total adversarial train accuarcy: 53.31
Total adversarial train loss:


Current batch: 20
Current benign test accuracy: 0.72
Current benign test loss: 0.7688397169113159
Current adversarial test accuracy: 0.41
Current adversarial test loss: 1.3153892755508423

Current batch: 30
Current benign test accuracy: 0.74
Current benign test loss: 0.7821694016456604
Current adversarial test accuracy: 0.47
Current adversarial test loss: 1.39421808719635

Current batch: 40
Current benign test accuracy: 0.76
Current benign test loss: 0.7421520352363586
Current adversarial test accuracy: 0.46
Current adversarial test loss: 1.4173245429992676

Current batch: 50
Current benign test accuracy: 0.83
Current benign test loss: 0.776124894618988
Current adversarial test accuracy: 0.43
Current adversarial test loss: 1.5018190145492554

Current batch: 60
Current benign test accuracy: 0.71
Current benign test loss: 0.8568723201751709
Current adversarial test accuracy: 0.42
Current adversarial test loss: 1.5095820426940918

Current batch: 70
Current benign test accuracy: 0.77
Curr

In [10]:
#RTX3070
import datetime
deltatime = str(datetime.timedelta(seconds=t2-t1))
print('Total running time is %s '% deltatime)

Total running time is 4:24:19.248712 
